## Proses Text Normalization

In [3]:
import pandas as pd
import numpy as np
import re
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import word_tokenize
import time



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv',)
df_train.head()

,text,label,pro_text_no_angka,no_stopwords
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini milik oleh usaha pabrik tahu yang s...,warung milik usaha pabrik puluh kenal putih ba...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k mmbri hujjah partai...,mohon ulama lurus k mmbri hujjah partai diwlh ...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatra bandung temp...,lokasi strategis jalan sumatra bandung nya nya...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya unboxing paket barang nya b...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,aduh jadi mahasiswa jangan sombong dong kasih ...,aduh mahasiswa sombong kasih kartu kuning ajar...


In [9]:
df_train.shape, df_valid.shape, df_test.shape

((11000, 4), (1260, 3), (500, 3))

In [15]:
print('train :\n',df_train.label.value_counts(),'\n')
print('validation :\n',df_valid.label.value_counts(),'\n')
print('test :\n',df_test.label.value_counts(),'\n')

train :
 positive    6416
negative    3436
neutral     1148
Name: label, dtype: int64 

validation :
 positive    735
negative    394
neutral     131
Name: label, dtype: int64 

test :
 positive    208
negative    204
neutral      88
Name: label, dtype: int64 



In [24]:
88/500

0.176

In [7]:
df_valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
df_valid.head()

,text,label,ori_text
0,meski masa kampanye sudah selesai bukan arti h...,neutral,"meski masa kampanye sudah selesai , bukan bera..."
1,tidak enak,negative,tidak enak
2,restoran ini tawar makan sunda kami mes ayam g...,positive,restoran ini menawarkan makanan sunda . kami m...
3,lokasi di alun alun masakan padang ini cukup k...,positive,lokasi di alun alun masakan padang ini cukup t...
4,betapa bejat kader gerindra yang anggota dewan...,negative,betapa bejad kader gerindra yang anggota dprd ...


In [6]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')
df_test.head()

,text,label,ori_text
0,kemarin gue datang ke tempat makan baru yang a...,negative,kemarin gue datang ke tempat makan baru yang a...
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,kayak nya sih gue tidak akan mau balik lagi ke...
2,kalau pikir benar tidak ada yang bisa bangga d...,negative,"kalau dipikir-pikir , sebenarnya tidak ada yan..."
3,ini pertama kali gue ke bank buat urus buat re...,negative,ini pertama kalinya gua ke bank buat ngurusin ...
4,waktu sampai dengan gue pernah suruh ibu latih...,negative,waktu sampai dengan gue pernah disuruh ibu lat...


In [ ]:
kamus_alay = pd.read_csv('data/new_kamusalay.csv', encoding='Latin-1',names=('alay','baku'))
kamus_alay.head()

,alay,baku
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [ ]:

# Melakukan Cleaning tweet tanpa menghapus stopwords
def text_proses(text):

    # Untuk stemmer kata
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    if type(text) == np.float64:
        return ""
    temp = re.sub(r'\b(USER|RT|URL)\b', '', text)
    temp = temp.lower()
    temp = re.sub(r"@[A-Za-z0-9_]+","", temp) #Menghilangkan Mention
    temp = re.sub("#[A-Za-z0-9_]+","", temp) #Menghilangkan Hashtag
    temp = re.sub(r'https://\S+', '', temp)
    # Melakukan subtitusi kata alay dengan kata baku
    temps = temp.split()
    for i in range(len(temps)):
        temp = temps[i]
        replacement = kamus_alay[kamus_alay['alay'] == temp]['baku']
        if not replacement.empty:
            temps[i] = replacement.iloc[0]
    temp = " ".join(temp.strip() for temp in temps)
    # temp = " ".join(temp.strip() for temp in temps if temp not in stopwords)

    # Mengubah angka menjadi terbaca
    # temps = temp.split()
    # for i in range(len(temps)):
    #     temp = temps[i]
    #     if temps[i].isdigit() == True :
    #         temps[i] = Terbilang().parse(temps[i]).getresult()
    #     else :
    #         temps[i]
    # temp = " ".join(temps)

    # Melakukan stemming
    tokens = word_tokenize(temp)
    temp = ' '.join([word for word in tokens ])
    temp = stemmer.stem(temp)

    temp = re.sub(r'\\x..', '', temp) #Menghilangkan Emoticon
    temp = re.sub(r'&amp;', 'dan', temp)
    temp = re.sub(r'http\S+', '', temp) #Menghilangkan link
    temp = re.sub(r"www.\S+", "", temp) #Menghilangkan link
    temp = re.sub(r"\\[n|t]", "", temp) #Menghilangkan 'enter' dan 'tab'
    temp = re.sub("[^A-Za-z\s']","", temp) #Menghilangkan yang bukan huruf
    temps = temp.split()
    temp = " ".join(temp.strip() for temp in temps)

    return temp

# kamus_alay = pd.read_csv('data/new_kamusalay.csv', encoding = 'Latin-1', header= None, names=['Alay','Baku'])


In [ ]:
start_ = time.time()
df_train.loc[:,'pro_text'] = df_train.loc[:,'text'].apply(lambda x : text_proses(x)).copy()
end_ = time.time()
print(f'Lama proses cleaning : {end_ - start_} detik')

print(f'Lama proses cleaning : {(end_ - start_)/60} menit')

Lama proses cleaning : 2953.822429895401 detik
Lama proses cleaning : 49.230373831590015 menit


In [26]:
df_train.head()

,text,label,pro_text_no_angka,no_stopwords
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini milik oleh usaha pabrik tahu yang s...,warung milik usaha pabrik puluh kenal putih ba...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k mmbri hujjah partai...,mohon ulama lurus k mmbri hujjah partai diwlh ...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatra bandung temp...,lokasi strategis jalan sumatra bandung nya nya...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya unboxing paket barang nya b...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,aduh jadi mahasiswa jangan sombong dong kasih ...,aduh mahasiswa sombong kasih kartu kuning ajar...


## Jumlah character pada cuitan sebelum normalisasi + setelah normalisasi

In [27]:
# Melihat jumlah karakter (max, min, mean) sebelum normalisasi
np.max(df_train.text.apply(len)), np.min(df_train.text.apply(len)), np.mean(df_train.text.apply(len))

(567, 3, 188.8969090909091)

In [29]:
# Melihat jumlah karakter (max, min, mean) setelah normalisasi
np.max(df_train.pro_text_no_angka.apply(len)), np.min(df_train.pro_text_no_angka.apply(len)), np.mean(df_train.pro_text_no_angka.apply(len))

(511, 3, 169.5508181818182)

## Jumlah Kata pada cuitan sebelum normalisasi + setelah normalisasi

In [28]:
# Melihat jumlah kata (max, min, mean) sebelum normalisasi
np.max(df_train.text.apply(lambda x: len(x.split()))), np.min(df_train.text.apply(lambda x: len(x.split()))), np.mean(df_train.text.apply(lambda x: len(x.split())))

(110, 1, 32.93127272727273)

In [30]:
# Melihat jumlah kata (max, min, mean) setelah normalisasi
np.max(df_train.pro_text_no_angka.apply(lambda x: len(x.split()))), np.min(df_train.pro_text_no_angka.apply(lambda x: len(x.split()))), np.mean(df_train.pro_text_no_angka.apply(lambda x: len(x.split())))

(85, 1, 29.05390909090909)

## Melihat Most words use in data

### Seluruh data

In [35]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df_train.pro_text_no_angka)
print(len(counter))
print(counter.most_common(10))

11877
[('nya', 13475), ('yang', 9339), ('dan', 7919), ('di', 7859), ('makan', 6493), ('tidak', 4518), ('saya', 4309), ('dengan', 4034), ('enak', 3730), ('tempat', 3605)]


## Data Positive

In [38]:
counter_positive = counter_word(df_train[df_train.label=='positive'].pro_text_no_angka)
print(len(counter_positive))
print(counter_positive.most_common(10))

7936
[('nya', 10919), ('yang', 7379), ('dan', 6434), ('di', 6277), ('makan', 5758), ('enak', 3463), ('tempat', 3270), ('dengan', 3258), ('saya', 3098), ('ini', 2717)]


### Data Negative

In [39]:
counter_negative = counter_word(df_train[df_train.label=='negative'].pro_text_no_angka)
print(len(counter_negative))
print(counter_negative.most_common(10))

5949
[('nya', 2350), ('tidak', 2023), ('yang', 1740), ('dan', 1257), ('di', 1241), ('saya', 1082), ('ini', 778), ('makan', 700), ('saja', 697), ('dengan', 643)]


### Data Neutral

In [40]:
counter_neutral = counter_word(df_train[df_train.label=='neutral'].pro_text_no_angka)
print(len(counter_neutral))
print(counter_neutral.most_common(10))

3740
[('di', 341), ('dan', 228), ('yang', 220), ('nya', 206), ('indonesia', 157), ('untuk', 146), ('pilih', 135), ('dengan', 133), ('saya', 129), ('partai', 128)]


## Melihat Most words use in data + stopwords

In [49]:
df_train.no_stopwords.isna().sum()

0

In [48]:

df_train.dropna(subset=['no_stopwords'],inplace=True,)

In [50]:
counterstop = counter_word(df_train.no_stopwords)
print(len(counterstop))
print(counterstop.most_common(10))

11515
[('nya', 13475), ('makan', 6493), ('enak', 3730), ('harga', 2022), ('bandung', 1753), ('menu', 1720), ('banget', 1523), ('restoran', 1294), ('pilih', 1284), ('layan', 1274)]


### Data Positive

In [51]:
counterstop_positive = counter_word(df_train[df_train.label=='positive'].no_stopwords)
print(len(counterstop_positive))
print(counterstop_positive.most_common(10))

7593
[('nya', 10919), ('makan', 5758), ('enak', 3463), ('harga', 1720), ('bandung', 1638), ('menu', 1573), ('banget', 1202), ('restoran', 1159), ('pilih', 959), ('suasana', 936)]


### Data negative

In [52]:
counterstop_negative = counter_word(df_train[df_train.label=='negative'].no_stopwords)
print(len(counterstop_negative))
print(counterstop_negative.most_common(10))

5623
[('nya', 2350), ('makan', 700), ('orang', 470), ('layan', 330), ('indonesia', 323), ('banget', 318), ('sih', 308), ('harga', 278), ('enak', 263), ('mahal', 255)]


### Data Neutral

In [53]:
counterstop_neutral = counter_word(df_train[df_train.label=='neutral'].no_stopwords)
print(len(counterstop_neutral))
print(counterstop_neutral.most_common(10))

3463
[('nya', 206), ('indonesia', 157), ('pilih', 135), ('partai', 128), ('demokrat', 102), ('daerah', 98), ('gubernur', 95), ('wakil', 68), ('kepala', 67), ('bank', 60)]
